Work in Progress

**Installing** dependency

we are keeping for now all the file of ALBERT in transformers.jl (it will Ultimately be added in TextAnalysis.jl with Transformers.jl as its dependency) we are using tokenize and Wordpiece(sentencepiece) from Albert which is based on Transformers.jl

In [21]:
using Transformers

Here we are taking 3 sentence as sample

In [22]:
sample1 = "God is Great! I won a lottery."
sample2 = "If all their conversations in the three months he had been coming to the diner were put together, it was doubtful that they would make a respectable paragraph."
sample3 = "She had the job she had planned for the last three years."
sample = [sample1,sample2,sample3]


3-element Array{String,1}:
 "God is Great! I won a lottery."                                                                                                                                 
 "If all their conversations in the three months he had been coming to the diner were put together, it was doubtful that they would make a respectable paragraph."
 "She had the job she had planned for the last three years."                                                                                                      

ALBERT uses SentencePiece for word segmentation and Sentence segementation.
Here we are going to provide both Sentencepiece as well as Wordpiece (using Vocab file of ALBERT) for ALBERT with the former as defalut one.
We first demonstrate WordPiece
To use word piece we need to make following changes as below


we first replace " "(space) with ' _' [space(U+2581)]

To decode split sentence. SentencePiece replace space with U+2581 and Store word in .vocab file with U+2581 as prefix .To tackle this we are using little trick as below.

In [23]:
We will replace it with better implementation of SentencePiece 

LoadError: syntax: extra token "will" after end of expression

In [24]:
sample[1] = replace(sample[1]," "=> " ▁")
sample[2] = replace(sample[2]," "=> " ▁")
sample[3] = replace(sample[3]," "=> " ▁")


"She ▁had ▁the ▁job ▁she ▁had ▁planned ▁for ▁the ▁last ▁three ▁years."

We Can use WordPiece for Word Segmentation with ALBERT vocabulary as follow. 
working on the APIs for Wordpiece

In [25]:
sample

3-element Array{String,1}:
 "God ▁is ▁Great! ▁I ▁won ▁a ▁lottery."                                                                                                                                                      
 "If ▁all ▁their ▁conversations ▁in ▁the ▁three ▁months ▁he ▁had ▁been ▁coming ▁to ▁the ▁diner ▁were ▁put ▁together, ▁it ▁was ▁doubtful ▁that ▁they ▁would ▁make ▁a ▁respectable ▁paragraph."
 "She ▁had ▁the ▁job ▁she ▁had ▁planned ▁for ▁the ▁last ▁three ▁years."                                                                                                                      

In [26]:
using Transformers.BidirectionalEncoder

In [27]:
tokens = tokenise.(sample, Val(true))


3-element Array{Array{String,1},1}:
 ["god", "▁is", "▁great", "!", "▁i", "▁won", "▁a", "▁lottery", "."]                                                                                                                                 
 ["if", "▁all", "▁their", "▁conversations", "▁in", "▁the", "▁three", "▁months", "▁he", "▁had"  …  "▁was", "▁doubtful", "▁that", "▁they", "▁would", "▁make", "▁a", "▁respectable", "▁paragraph", "."]
 ["she", "▁had", "▁the", "▁job", "▁she", "▁had", "▁planned", "▁for", "▁the", "▁last", "▁three", "▁years", "."]                                                                                      

In [28]:
vocab = readlines("/home/iamtejas/Downloads/albert_xlarge_v1/albert_xlarge/30k-clean.vocab")
vocabnew = split.(vocab , "\t")
vo = [vocabnew[1][1]]


1-element Array{SubString{String},1}:
 "<pad>"

We don't need Score of SentencePiece unigram model for Wordpiece.

In [29]:
for i in 1:30000
    vocab1 = vocabnew[i][1]
    push!(vo,vocab1)
end
vocab1 = convert(Array{String,1},vo)

30001-element Array{String,1}:
 "<pad>"        
 "<pad>"        
 "<unk>"        
 "[CLS]"        
 "[SEP]"        
 "[MASK]"       
 "("            
 ")"            
 "\""           
 "-"            
 "."            
 "–"            
 "£"            
 ⋮              
 "▁predation"   
 "▁saviour"     
 "▁archivist"   
 "▁obverse"     
 "error"        
 "▁tyrion"      
 "▁addictive"   
 "▁veneto"      
 "▁colloquial"  
 "agog"         
 "▁deficiencies"
 "▁eloquent"    

In [30]:
vocab1 = vocab1[2:30001]

30000-element Array{String,1}:
 "<pad>"        
 "<unk>"        
 "[CLS]"        
 "[SEP]"        
 "[MASK]"       
 "("            
 ")"            
 "\""           
 "-"            
 "."            
 "–"            
 "£"            
 "€"            
 ⋮              
 "▁predation"   
 "▁saviour"     
 "▁archivist"   
 "▁obverse"     
 "error"        
 "▁tyrion"      
 "▁addictive"   
 "▁veneto"      
 "▁colloquial"  
 "agog"         
 "▁deficiencies"
 "▁eloquent"    

In [31]:
wp = WordPiece(vocab1,"<unk>"; max_char = 200)

WordPiece(vocab_size=30000, unk=<unk>, max_char=200)

In [32]:
tokensnew = wp.(tokens)

3-element Array{Array{String,1},1}:
 ["god", "▁is", "▁great", "!", "▁i", "▁won", "▁a", "▁lottery", "."]                                                                                                                                 
 ["if", "▁all", "▁their", "▁conversations", "▁in", "▁the", "▁three", "▁months", "▁he", "▁had"  …  "▁was", "▁doubtful", "▁that", "▁they", "▁would", "▁make", "▁a", "▁respectable", "▁paragraph", "."]
 ["she", "▁had", "▁the", "▁job", "▁she", "▁had", "▁planned", "▁for", "▁the", "▁last", "▁three", "▁years", "."]                                                                                      

In [33]:
sample

3-element Array{String,1}:
 "God ▁is ▁Great! ▁I ▁won ▁a ▁lottery."                                                                                                                                                      
 "If ▁all ▁their ▁conversations ▁in ▁the ▁three ▁months ▁he ▁had ▁been ▁coming ▁to ▁the ▁diner ▁were ▁put ▁together, ▁it ▁was ▁doubtful ▁that ▁they ▁would ▁make ▁a ▁respectable ▁paragraph."
 "She ▁had ▁the ▁job ▁she ▁had ▁planned ▁for ▁the ▁last ▁three ▁years."                                                                                                                      

In [34]:
vocabalbert = Transformers.Basic.Vocabulary(wp.vocab,wp.vocab[wp.unk_idx])

Vocabulary(30000, unk=<unk>)

here we have indices form sample tokens.


In [35]:
indices = vocabalbert(tokensnew)

30×3 Array{Int64,2}:
  5475    822  1080
    26     66    42
   375     67    15
   188  13528  1206
    32     20    40
   231     15    42
    22    133  2036
 17519    819    27
    10     25    15
     2     42   237
     2     75   133
     2    881   123
     2     21    10
     ⋮             
     2     16     2
     2     33     2
     2     24     2
     2  22569     2
     2     31     2
     2     60     2
     2     84     2
     2    234     2
     2     22     2
     2  22740     2
     2  20600     2
     2     10     2

In [36]:
size(indices)

(30, 3)

since we only take one sentence as input

In [37]:
seg_indices = ones(Int, size(indices)...)

30×3 Array{Int64,2}:
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 ⋮      
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1

Next we need to turn those indices into embeddings.These can be done by loading embedding in Embed 

## SENTENCEPIECE
now i will show how to use Sentencepiece  
I am working on julia implementation of SentencePiece but for now we can use PyCall and also python Wrapper is under development


### Environment setup
You can install Python binary package of SentencePiece with.

> % pip install sentencepiece

For detail visit - [Sentencpiece](https://github.com/google/sentencepiece)


In [38]:
using PyCall
SentencePiece = pyimport("sentencepiece")
sp = SentencePiece.SentencePieceProcessor()
sp.Load("/home/iamtejas/Downloads/albert_xlarge_v1/albert_xlarge/30k-clean.model")


true

#### Avaliable APIs
> sp.EncodeAsPieces

> sp.EncodeAsIds

In [39]:
sample4 = "God is Great! I won a lottery."
sample5 = "If all their conversations in the three months he had been coming to the diner were put together, it was doubtful that they would make a respectable paragraph."
sample6 = "She had the job she had planned for the last three years."
sampleforsp = [sample4,sample5,sample6]

3-element Array{String,1}:
 "God is Great! I won a lottery."                                                                                                                                 
 "If all their conversations in the three months he had been coming to the diner were put together, it was doubtful that they would make a respectable paragraph."
 "She had the job she had planned for the last three years."                                                                                                      

In [40]:
sp.EncodeAsPieces(sampleforsp[1])

15-element Array{String,1}:
 "▁"       
 "G"       
 "od"      
 "▁is"     
 "▁"       
 "G"       
 "re"      
 "at"      
 "!"       
 "▁"       
 "I"       
 "▁won"    
 "▁a"      
 "▁lottery"
 "."       

In [41]:
E1 = sp.EncodeAsIds(sampleforsp[1])

15-element Array{Int64,1}:
    13
     1
  5648
    25
    13
     1
    99
   721
   187
    13
     1
   230
    21
 17518
     9

In [42]:
E2 = sp.EncodeAsIds(sampleforsp[2])

32-element Array{Int64,1}:
    13
     1
   410
    65
    66
 13527
    19
    14
   132
   818
    24
    41
    74
     ⋮
    15
    32
    23
 22568
    30
    59
    83
   233
    21
 22739
 20599
     9

In [43]:
E3 = sp.EncodeAsIds(sampleforsp[3])

15-element Array{Int64,1}:
   13
    1
  438
   41
   14
 1205
   39
   41
 2035
   26
   14
  236
  132
  122
    9

In [44]:
E1 = [E1;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0]


32-element Array{Int64,1}:
   13
    1
 5648
   25
   13
    1
   99
  721
  187
   13
    1
  230
   21
    ⋮
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0

In [45]:
E3 = [E3;;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0]

32-element Array{Int64,1}:
   13
    1
  438
   41
   14
 1205
   39
   41
 2035
   26
   14
  236
  132
    ⋮
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0

if we are going to feed all the three sentence in the model we need to add pad token for all the input we for now we can to do it manually later will be automatically can be provided


In [46]:
E = [E1 E2 E3]

32×3 Array{Int64,2}:
   13     13    13
    1      1     1
 5648    410   438
   25     65    41
   13     66    14
    1  13527  1205
   99     19    39
  721     14    41
  187    132  2035
   13    818    26
    1     24    14
  230     41   236
   21     74   132
    ⋮             
    0     15     0
    0     32     0
    0     23     0
    0  22568     0
    0     30     0
    0     59     0
    0     83     0
    0    233     0
    0     21     0
    0  22739     0
    0  20599     0
    0      9     0

In [47]:
seg_idx = ones(Int, size(indices)...)

30×3 Array{Int64,2}:
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 ⋮      
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1

Now its time to use released weight 

In [48]:
using Transformers.Basic
using Flux
using Flux: loadparams!

In [49]:
using BSON: @save, @load
@load "/home/iamtejas/Downloads/albert_base_v1.bson.tfbson" config weights vocab

In [50]:
tok_emb = Embed(
    config["embedding_size"],
    config["vocab_size"]
  )

  seg_emb = Embed(
    config["embedding_size"],
    config["type_vocab_size"]
  )

  posi_emb = PositionEmbedding(
    config["embedding_size"],
    config["max_position_embeddings"];
    trainable = true
  )

 emb_post = Positionwise(
    LayerNorm(
      config["embedding_size"]
    ),
       # Dropout(
       #     config["hidden_dropout_prob"]
       # ) there is some problem in loading dropout
  )

embedding = Dict{Symbol, Any}()

Dict{Symbol,Any} with 0 entries

In [51]:
vnames = keys(weights)
embeddings_weights = filter(name->occursin("embeddings", name), vnames)

Set(["bert/embeddings/token_type_embeddings", "bert/embeddings/LayerNorm/gamma", "bert/embeddings/word_embeddings", "bert/embeddings/LayerNorm/beta", "bert/embeddings/position_embeddings"])

In [52]:
 for k ∈ embeddings_weights
        if occursin("LayerNorm/gamma", k)
            loadparams!(emb_post[1].diag.α', [weights[k]]) #there is some problem with loading
            embedding[:postprocessor] = emb_post
        elseif occursin("LayerNorm/beta", k)
            loadparams!(emb_post[1].diag.β', [weights[k]])
        elseif occursin("word_embeddings", k)
            loadparams!(tok_emb.embedding, [weights[k]])
            embedding[:tok] = tok_emb
        elseif occursin("position_embeddings", k)
            loadparams!(posi_emb.embedding, [weights[k]])
            embedding[:pe] = posi_emb
        elseif occursin("token_type_embeddings", k)
             loadparams!(seg_emb.embedding, [weights[k]])
            embedding[:segment] = seg_emb
        else
            @warn "unknown variable: $k"
        end
    end

In [53]:
 albertembed = CompositeEmbedding(;embedding...)

CompositeEmbedding(tok = Embed(128), segment = Embed(128), pe = PositionEmbedding(128, max_len=512), postprocessor = Positionwise(LayerNorm(128)))

In [54]:
embedding_wordpiece = albertembed(tok=indices, segment=seg_indices)

128×30×3 Array{Float32,3}:
[:, :, 1] =
 -0.591439   1.13699    -0.025718   …   0.831079   0.704506    0.440932 
 -0.799194   0.0305692  -0.438631      -0.12549    0.0807379   0.121085 
  2.17368   -0.469415    0.284542      -0.720768  -0.75624    -0.661375 
  2.52347    1.03309     0.92254        0.410215   0.167004    0.0673373
 -0.176611  -1.4789     -0.681304       1.38727    1.40006     1.2112   
  1.16329    1.35542     0.611702   …   1.28013    1.36642     1.50305  
 -1.98747   -1.66523     0.601882       2.24999    2.35954     2.23884  
 -0.310094   2.92642     0.415302       2.34161    2.22562     2.20565  
 -1.37891   -1.38538     0.118553       0.368      0.174424    0.377196 
  1.39736   -0.675551   -0.398878       1.09734    1.24373     0.998397 
 -1.81548   -0.950838   -0.368885   …  -1.78001   -2.07668    -2.44817  
  0.018042  -1.26416    -0.397725       1.02105    1.06319     1.03057  
 -1.21238    0.840682    0.598232      -1.14324   -1.12482    -1.15721  
  ⋮         

here we have embedding from wordPiece 

### Embedding from Sentencepiece

In [55]:
seg_idx_sentencepiece = ones(Int, size(E)...)

32×3 Array{Int64,2}:
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 ⋮      
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1

In [56]:
embedding_wordpiece = albertembed(tok=E, segment=seg_idx_sentencepiece)

128×32×3 Array{Float32,3}:
[:, :, 1] =
 -1.27957   -0.433449   -0.0945636  …  -0.49776    -1.01922     -1.46112  
 -0.57838   -0.980475   -1.04832       -0.267728   -0.358444    -0.669767 
  2.79966   -0.82644    -0.925393       0.128052    0.245785     0.351939 
 -0.709518  -0.429736   -1.00555       -0.254376    0.00938835   0.53717  
 -1.07688   -0.63944     0.818117       1.52909     1.14141      0.994143 
  1.06804   -0.364175   -2.47114    …   0.945338    0.898164     0.723984 
 -1.31039   -1.17958    -0.123138       1.88349     1.43553      0.911783 
  0.742485  -1.6949     -2.15704       -1.04646    -1.0591      -0.997114 
  0.160304  -1.65645    -0.76259       -1.20975    -0.545515     0.448567 
  1.53976    0.762963   -1.84859        2.10263     1.4055       0.538642 
 -1.72798   -0.458754   -1.34792    …   0.173283   -0.328897    -0.631871 
 -1.43214    0.546417    0.393653       2.23232     1.99174      1.72529  
  2.65905    1.69725     1.18012        0.418767    0.0581364

the above we have embedding from SentencePiece

### ALBERT TRANSFORMER WRAP

In [87]:
using Transformers
using Flux: @functor
using MacroTools: @forward

using Transformers.Basic
using Transformers.Basic: AbstractTransformer
using Transformers.Stacks

struct albert <: AbstractTransformer
  ts::Stack
end

@functor albert

@forward albert.ts Base.getindex, Base.length


function albert(size::Int, head::Int, ps::Int, layer::Int;
              act = gelu, attn_pdrop = 0)
  rem(size,  head) != 0 && error("size not divisible by head")
  albert(size, head, div(size, head), ps, layer; act=act, attn_pdrop=attn_pdrop)
end

function albert(size::Int, head::Int, hs::Int, ps::Int, layer::Int; act = gelu, attn_pdrop = 0)
  albert(
    Stack(
      @nntopo_str("((x, m) => x':(x, m)) => $layer"),
      [
        Transformer(size, head, hs, ps; future=true, act=act, pdrop=attn_pdrop)
        for i = 1:layer
      ]...
    )
         )
end

function (al::albert)(x::T, mask=nothing; all::Bool=false) where T
  e = x

  if mask === nothing
    t, ts = al.ts(e, nothing)
  else
    t, ts = al.ts(e, getmask(mask, mask))
  end

  if all
    if mask !== nothing
      ts = map(ts) do ti
        ti .* mask
      end
    end
    ts[end], ts
  else
    t = mask === nothing ? t : t .* mask
    t
  end
end

"""
    masklmloss(embed::Embed{T}, transform,
               t::AbstractArray{T, N}, posis::AbstractArray{Tuple{Int,Int}}, labels) where {T,N}
    masklmloss(embed::Embed{T}, transform, output_bias,
               t::AbstractArray{T, N}, posis::AbstractArray{Tuple{Int,Int}}, labels) where {T,N}

helper function for computing the maks language modeling loss.
Performance `transform(x) .+ output_bias` where `x` is the mask specified by
`posis`, then compute the similarity with `embed.embedding` and crossentropy between true `labels`.
"""
function masklmloss(embed::Embed{T}, transform, t::AbstractArray{T, N}, posis::AbstractArray{Tuple{Int,Int}}, labels) where {T,N}
  masktok = gather(t, posis)
  sim = logsoftmax(transpose(embed.embedding) * transform(masktok))
  return logcrossentropy(labels, sim)
end

function masklmloss(embed::Embed{T}, transform, output_bias, t::AbstractArray{T, N}, posis::AbstractArray{Tuple{Int,Int}}, labels) where {T,N}
  masktok = gather(t, posis)
  sim = logsoftmax(transpose(embed.embedding) * transform(masktok) .+ output_bias)
  return logcrossentropy(labels, sim)
end


function Base.show(io::IO, al::albert)
  hs = div(size(al.ts[1].mh.iqproj.W)[1], al.ts[1].mh.head)
  h, ps = size(al.ts[1].pw.dout.W)

  print(io, "albert(")
  print(io, "layers=$(length(al.ts)), ")
  print(io, "head=$(al.ts[1].mh.head), ")
  print(io, "head_size=$(hs), ")
  print(io, "pwffn_size=$(ps), ")
  print(io, "size=$(h))")
end

In [91]:
albert()

MethodError: MethodError: no method matching albert()
Closest candidates are:
  albert(!Matched::Int64, !Matched::Int64, !Matched::Int64, !Matched::Int64; act, attn_pdrop) at In[87]:20
  albert(!Matched::Int64, !Matched::Int64, !Matched::Int64, !Matched::Int64, !Matched::Int64; act, attn_pdrop) at In[87]:25
  albert(!Matched::Stack) at In[87]:10
  ...

In [92]:
function load_albert_from_tfbson(config, weights)
    #init albert model possible component
    albert = albert(
        config["hidden_size"],
        config["num_attention_heads"],
        config["intermediate_size"],
        config["num_hidden_layers"];
        act = get_activation(config["hidden_act"]),
        embedding =config["embedding_size"],
        pdrop = config["hidden_dropout_prob"],
        attn_pdrop = config["attention_probs_dropout_prob"]
    )
end

load_albert_from_tfbson (generic function with 1 method)

In [93]:
function get_activation(act_string)
    if act_string == "gelu"
        gelu
    elseif act_string == "relu"
        relu
    elseif act_string == "tanh"
        tanh
    elseif act_string == "linear"
        identity
    else
        throw(DomainError(act_string, "activation support: linear, gelu, relu, tanh"))
    end
end

get_activation (generic function with 1 method)

In [94]:
albert_model = load_albert_from_tfbson(config, weights)

UndefVarError: UndefVarError: albert not defined